In [14]:
import os
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.sql.functions import (
    regexp_replace,
    regexp_extract_all,
    col,
    lit
)

In [15]:
def create_spark_configuration() -> SparkConf:
    """
    Создает и конфигурирует экземпляр SparkConf для приложения Spark.

    Returns:
        SparkConf: Настроенный экземпляр SparkConf.
    """
    # Получаем имя пользователя
    user_name = os.getenv("USER")
    
    conf = SparkConf()
    conf.setAppName("lab 1 Test")
    conf.setMaster("yarn")
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.memory", "12g")
    conf.set("spark.executor.cores", "8")
    conf.set("spark.executor.instances", "2")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "2")
    conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}/warehouse")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf

In [16]:
conf = create_spark_configuration()

In [17]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

24/11/17 21:24:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [18]:
path = "/user/user1/2019oct.csv"

In [19]:
df = (spark.read.format("csv")
      .option("header", "true")
      .load(path)
)

In [20]:
df.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    NULL| 543.10|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:...|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:...|      view|   1004237|205301355563188265

In [24]:
df = df.select(
    "event_type", "product_id","category_code", "brand", "price"
)

In [25]:
df.show()

+----------+----------+--------------------+--------+-------+
|event_type|product_id|       category_code|   brand|  price|
+----------+----------+--------------------+--------+-------+
|      view|  44600062|                NULL|shiseido|  35.79|
|      view|   3900821|appliances.enviro...|    aqua|  33.20|
|      view|  17200506|furniture.living_...|    NULL| 543.10|
|      view|   1307067|  computers.notebook|  lenovo| 251.74|
|      view|   1004237|electronics.smart...|   apple|1081.98|
|      view|   1480613|   computers.desktop|  pulser| 908.62|
|      view|  17300353|                NULL|   creed| 380.96|
|      view|  31500053|                NULL|luminarc|  41.16|
|      view|  28719074|  apparel.shoes.keds|   baden| 102.71|
|      view|   1004545|electronics.smart...|  huawei| 566.01|
|      view|   2900536|appliances.kitche...|elenberg|  51.46|
|      view|   1005011|electronics.smart...| samsung| 900.64|
|      view|   3900746|appliances.enviro...|   haier| 102.38|
|      v

In [26]:
df.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)



In [27]:
def transform_dataframe(data: DataFrame) -> DataFrame:
    """
    Преобразует столбцы DataFrame в указанные типы данных и
    выполняет необходимые преобразования.

    Args:
        data (DataFrame): Исходный DataFrame.

    Returns:
        DataFrame: Преобразованный DataFrame.
    """
    # Преобразуем столбцы в соответствующие типы данных
    data = data.withColumn("product_id",
                           col("product_id").cast("Integer"))
    data = data.withColumn("price",
                           col("price").cast("Float"))


    return data

In [28]:
df = transform_dataframe(df)

In [29]:
df.show()

+----------+----------+--------------------+--------+-------+
|event_type|product_id|       category_code|   brand|  price|
+----------+----------+--------------------+--------+-------+
|      view|  44600062|                NULL|shiseido|  35.79|
|      view|   3900821|appliances.enviro...|    aqua|   33.2|
|      view|  17200506|furniture.living_...|    NULL|  543.1|
|      view|   1307067|  computers.notebook|  lenovo| 251.74|
|      view|   1004237|electronics.smart...|   apple|1081.98|
|      view|   1480613|   computers.desktop|  pulser| 908.62|
|      view|  17300353|                NULL|   creed| 380.96|
|      view|  31500053|                NULL|luminarc|  41.16|
|      view|  28719074|  apparel.shoes.keds|   baden| 102.71|
|      view|   1004545|electronics.smart...|  huawei| 566.01|
|      view|   2900536|appliances.kitche...|elenberg|  51.46|
|      view|   1005011|electronics.smart...| samsung| 900.64|
|      view|   3900746|appliances.enviro...|   haier| 102.38|
|      v

In [30]:
df.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)



In [31]:
database_name = "lopin_database1"

In [32]:
create_database_sql = f"""
CREATE DATABASE IF NOT EXISTS spark_catalog.{database_name}
"""

In [33]:
spark.sql(create_database_sql)

DataFrame[]

In [34]:
spark.catalog.setCurrentDatabase(database_name)

In [35]:
# Сохранение DataFrame в виде таблицы
df.writeTo("sobd_lab1_table").using("iceberg").create()

In [36]:
for table in spark.catalog.listTables():
    print(table.name)

sobd_lab1_table


In [37]:
spark.stop()